<a href="https://colab.research.google.com/github/Drfengze/ANN-CBIR/blob/main/1.2_training_dataset_corn_sentinelonly_june1_julyend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data preparation

In [3]:
import ee

ee.Authenticate()
ee.Initialize()
import numpy as np
from datetime import datetime, timedelta, date
import random
import pandas as pd


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=E5e-UDy9vctwT_RjKShEqCMSy-IKgp9TxXmu7zzsA2A&tc=P76v8U9DPuYRT-7ZT-brl5PQX9YM5QYMmzSIjQlZ8Ls&cc=Fn5h7WzRlU4ZbkMTIngyWWXhxbG8frQ0FzmHfpAsmnM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BVruYXuBEHkrAhN0mljlbfMdvg6dUde1SKtKZYMlUwqh-wbASGUBeM

Successfully saved authorization token.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# Cloud masking function.
def maskL8sr(image):
    cloudShadowBitMask = ee.Number(2).pow(4).int()
    cloudsBitMask = ee.Number(2).pow(3).int()
    qa = image.select("QA_PIXEL")
    mask1 = (
        qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    )
    mask2 = image.mask().reduce("min")
    # mask3 = image.select(opticalBands).gt(0).And(
    #     image.select(opticalBands).lt(10000)).reduce('min')
    mask = mask1.And(mask2)  # .And(mask3)
    return image.updateMask(mask)


# Cloud masking function for s2
# def maskS2clouds(image):
#     qa = image.select('QA60')
#     # Bits 10 and 11 are clouds and cirrus, respectively.
#     cloudBitMask = 1 << 10
#     cirrusBitMask = 1 << 11
#     # Both flags should be set to zero, indicating clear conditions.
#     mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
#         .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

#     return image.updateMask(mask)


def merge_s2_l8(s2, l8):
    merged = ee.ImageCollection([s2, l8]).mean()
    return merged

def filter_type(cdl):
    return cdl.eq(1).selfMask().multiply(1).unmask().add(cdl.eq(5).selfMask().multiply(2).unmask())

def img_vi(img):
    img = img.addBands(img.normalizedDifference(["nir", "red"]).rename("ndvi"))
    img = img.addBands(img.select(["ndvi"]).multiply(img.select(["nir"])).rename("nirv"))
    return img

In [ ]:
import ee

s2Sr = ee.ImageCollection('COPERNICUS/S2_SR')
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

# START_DATE = '2019-01-01'
# END_DATE = '2019-03-01'
# MAX_CLOUD_PROBABILITY = 65
# region = ee.Geometry.Rectangle([-76.5, 2.0, -74, 4.0], geodesic=False)

def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

def maskEdges(s2_img):
    return s2_img.updateMask(s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

# Filter input collections by desired data range and region.

s2Sr = s2Sr.filter(criteria).map(maskEdges)
s2Clouds = s2Clouds.filter(criteria)

# Join S2 SR with cloud probability dataset to add cloud mask.
s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
    primary=s2Sr,
    secondary=s2Clouds,
    condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
)

s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskClouds).median()
rgbVis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

# Use Map.addLayer in Python API to add this layer to your interactive map


In [40]:
bandNamesOut = ["blue", "green", "red", "nir", "swir1", "swir2"]
bandNamesl8 = ["SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]
bandNamesS2 = ["B2", "B3", "B4", "B8", "B11", "B12"]
BANDS = ["blue", "green", "red", "nir", "swir1", "swir2", "ndvi", "nirv"]
RESPONSE = "cropland"
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 64
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
MAX_CLOUD_PROBABILITY = 65
l8sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").map(maskL8sr)
l8sr = l8sr.select(bandNamesl8, bandNamesOut)
s2Sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
# df = pd.read_csv('/content/drive/MyDrive/RDA/model_and_env/calendar.csv')
country = 'United States of America'
crop = 'Corn Soybean'
# calendar = df.loc[(df['Country'] == country) & (df['Crop'] == crop)].to_dict(orient='records')[0]
us = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1").filter(
    'ADM0_NAME == "{}"'.format(country)
)
states = ['Iowa', 'Illinois', 'Indiana', 'Michigan', 'Ohio', 'Nebraska', 'Kansas', 'Minnesota', 'Missouri']
trainingPolys = us.filter(ee.Filter.inList("ADM1_NAME", states))
trainingPolysList = trainingPolys.toList(trainingPolys.size())

"""# Data Preparation"""
FOLDER = "{}_{}".format(crop,country).replace(' ','_')
TRAINING_BASE = "training_{}".format(crop).replace(' ','_')
EVAL_BASE = "eval_patches"

In [31]:
class DATA_EXPORTER:
  def __init__(self, image, cdl, kernel, randomseed, binary,scale,FOLDER):
      self.image = image
      self.cdl = cdl
      self.kernel = kernel
      self.randomseed = randomseed
      self.binary = binary
      # self.classcode = classcode
      self.folder = FOLDER
      self.scale = scale


  def export(self):
      wheat_lc = filter_type(self.cdl)
      mask = wheat_lc.neq(0)
      if self.binary:
          featureStack = ee.Image.cat([self.image, wheat_lc.select("cropland")]).float()
          # self.cdl_export(wheat_lc,self.description,self.trainingPoly)
      else:
          featureStack = ee.Image.cat([self.image, self.cdl.select("cropland")]).float()
          # self.cdl_export(self.cdl,self.description,self.trainingPoly)
      arrays = featureStack.neighborhoodToArray(self.kernel)
      arrays_masked = arrays.updateMask(mask)
      return arrays,wheat_lc

  def train_export(self,trainingPoly,desc,n=200):
      '''
      arrays: image collection
      trainingPoly: polygon to sample
      n: number of samples
      folder: folder to save to
      desc: description of task
      i: seed
      '''
      arrays,_ = self.export()
      # trainingPoly = trainingPoly.geometry().simplify(1)
      geomSample = ee.FeatureCollection([])
      for j in np.arange(n):
          sample = arrays.sample(
              region=trainingPoly,
              numPixels=5,  # Size of the shard.
              seed=random.randint(0, 10*n),
              scale=self.scale,
              tileScale=8,
          )
          geomSample = geomSample.merge(sample)

      task = ee.batch.Export.table.toDrive(
          folder=self.folder,
          collection=geomSample,
          description='train_'+desc,
          fileFormat="TFRecord",
      )
      task.start()


In [41]:
def process_date_range(start_date, end_date, s2Sr, l8sr,l8=False):
    s2_criteria = ee.Filter.date(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))
    l8_criteria = ee.Filter.date(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))

    s2_reduced = s2Sr.filter(s2_criteria)
    s2_reduced = s2_reduced.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',MAX_CLOUD_PROBABILITY)).map(maskS2clouds)
    s2_reduced = s2_reduced.select(bandNamesS2, bandNamesOut).median().divide(10000).float()
    l8_reduced = l8sr.filter(l8_criteria).median().multiply(0.0000275).add(-0.2).float()
    if l8:
      image = merge_s2_l8(s2_reduced, l8_reduced)
    else:
      image = s2_reduced
    return img_vi(image).float()  # 需要定义img_vi函数

def get_date_range(YEAR, month, half):
    start_date = date(YEAR, month, 1 + 15 * half)
    if half == 0:
        end_date = date(YEAR, month, 15)
    else:
        end_date = date(YEAR, month, 30) if month == 6 else date(YEAR, 8, 1)
    return start_date, end_date

def handle_year(YEAR, s2Sr, l8sr):
    image_combined = None

    for month in [6, 7]:
        for half in range(2):
            start_date, end_date = get_date_range(YEAR, month, half)
            image = process_date_range(start_date, end_date, s2Sr, l8sr)
            if image_combined is None:
                image_combined = image
            else:
                image_combined = image_combined.addBands(image)

    return image_combined

In [43]:
binary = True
FOLDER = "{}_{}_{}".format(crop,country,str(binary)).replace(' ','_')
list1 = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list1, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)
for YEAR in range(2017, 2018):
    image = handle_year(YEAR, s2Sr, l8sr)
    start_year = date(YEAR, 1, 1).strftime("%Y-%m-%d")
    end_year = '2022-12-31'
    cdl = ee.ImageCollection('USDA/NASS/CDL').filterDate(start_year, end_year).first().select('cropland')
    # These numbers determined experimentally.
    n = 200  # Number of shards in each polygon.
    N = 1000  # Total sample size in each polygon.
    # crop_code = int(calendar['Crop Code'])
    i = random.randint(0, 100)
    data_export = DATA_EXPORTER(image, cdl, kernel, i, binary,30,FOLDER)
    for g in states:
        trainingPoly = us.filter(ee.Filter.eq("ADM1_NAME", g))
        # run_task(arrays,trainingPoly,desc,i)
        if binary:
            desc = 'binary_{}_{}_{}'.format(crop,g,YEAR).replace(' ','_')
            data_export.train_export(trainingPoly,desc)
        else:
            desc = 'multiclass_{}_{}_{}'.format(crop,g,YEAR).replace(' ','_')
            data_export.train_export(trainingPoly,desc)



## Export task

In [29]:
from pprint import pprint

pprint(ee.batch.Task.list())

[<Task ZCQIFVV6INWAVZ5P23NM35Q3 EXPORT_FEATURES: train_binary_Corn_Soybean_Missouri_2017 (READY)>,
 <Task MTL2RKRD56L3CIRVM3Z7XKTI EXPORT_FEATURES: train_binary_Corn_Soybean_Minnesota_2017 (READY)>,
 <Task B75QEUPRJB2E6AKFU4VOOPN3 EXPORT_FEATURES: train_binary_Corn_Soybean_Kansas_2017 (READY)>,
 <Task NDLX43H2VMY2QT6PSVOCXW6J EXPORT_FEATURES: train_binary_Corn_Soybean_Nebraska_2017 (READY)>,
 <Task 4O47EYLZS44YN3I6A4B77HCO EXPORT_FEATURES: train_binary_Corn_Soybean_Ohio_2017 (READY)>,
 <Task IU5XMIBXTCGN64A6CTLG7FSS EXPORT_FEATURES: train_binary_Corn_Soybean_Michigan_2017 (READY)>,
 <Task IOJN3NSF5DKAL4652K7ONP7N EXPORT_FEATURES: train_binary_Corn_Soybean_Indiana_2017 (READY)>,
 <Task 7R7FCSFCYE6CKRYNDUJC37OH EXPORT_FEATURES: train_binary_Corn_Soybean_Illinois_2017 (READY)>,
 <Task UUECZA44S7DBKC7CV4ZVH642 EXPORT_FEATURES: train_binary_Corn_Soybean_Iowa_2017 (READY)>,
 <Task PD7SMUTQFUAAEG3K2LM5VLIV EXPORT_FEATURES: train_binary_Corn_Soybean_Missouri_2017 (COMPLETED)>,
 <Task KSD52R5FZ